In [7]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.datasets import mnist


print(f"Tensorflow {tf.__version__}")

Tensorflow 2.20.0


In [9]:

def cnn_three_blocks_ecpos_100():

    inputs = tf.keras.Input(shape=(28, 28, 1))

    # Block 1
    x = Conv2D(32, (3, 3), padding='same', activation='relu', use_bias=False)(inputs)
    x = Conv2D(32, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 3
    x = Conv2D(128, (3, 3), padding='same', activation='relu', use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    # Classifier
    x = Flatten()(x)
    x = Dense(256, use_bias=False)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(10, activation='softmax', use_bias=False)(x)


    # Create functional model
    model= Model(inputs, outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [8]:
# Download data
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [11]:
model = cnn_three_blocks_ecpos_100()

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=100,                            # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v2.keras")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 14, 14, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 14, 14, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 7, 7, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 436,000 (1.66 MB)

 Trainable params: 436,000 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 75s 172ms/step - accuracy: 0.1359 - loss: 2.2778 - val_accuracy: 0.6472 - val_loss: 2.1524
Epoch 2/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 75s 178ms/step - accuracy: 0.4872 - loss: 1.4853 - val_accuracy: 0.9122 - val_loss: 0.3976
Epoch 3/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 78s 185ms/step - accuracy: 0.8133 - loss: 0.5765 - val_accuracy: 0.9563 - val_loss: 0.1671
Epoch 4/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 86s 204ms/step - accuracy: 0.8827 - loss: 0.3747 - val_accuracy: 0.9640 - val_loss: 0.1257
Epoch 5/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 78s 186ms/step - accuracy: 0.9050 - loss: 0.3059 - val_accuracy: 0.9650 - val_loss: 0.1159
Epoch 6/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 78s 185ms/step - accuracy: 0.9213 - loss: 0.2578 - val_accuracy: 0.9732 - val_loss: 0.0916
Epoch 7/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 75s 177ms/step - accuracy: 0.9287 - loss: 0.2313 - val_accuracy: 0.9742 - val_loss: 0.0823
Epoch 8/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 76s 180ms/step - accuracy: 0.9

In [3]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [4]:
net = snn_converter.load_keras_model("CIFAR_v2.keras", show_info=True)

2025-11-06 11:17:41.924804: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 11:17:41.927148: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-06 11:17:42.201313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-06 11:17:44.204886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

* Input layer: input_layer_1, (28, 28, 1)
* InputLayer skipped.
* Conv2D layer: conv2d_5, (28, 28, 32) 
* Conv2D layer: conv2d_6, (28, 28, 32) 
* MaxPooling2D layer: max_pooling2d_3, (14, 14, 32) 
* Dropout skipped.
* Conv2D layer: conv2d_7, (14, 14, 64) 
* Conv2D layer: conv2d_8, (14, 14, 64) 
* MaxPooling2D layer: max_pooling2d_4, (7, 7, 64) 
* Dropout skipped.
* Conv2D layer: conv2d_9, (7, 7, 128) 
* MaxPooling2D layer: max_pooling2d_5, (3, 3, 128) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense_2, 256 
    weights: (256, 1152)
    mean -0.0006090263486839831, std 0.03802994266152382
    min -0.11883995682001114, max 0.1181797906756401
* Activation skipped.
* Dropout skipped.
* Dense layer: dense_3, 10 
    weights: (10, 256)
    mean -0.0012305330019444227, std 0.10003773868083954
    min -0.25657472014427185, max 0.20399850606918335



In [9]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [20:19<00:00,  4.07s/it]


In [10]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       1.00      1.00      1.00        41
           2       1.00      1.00      1.00        32
           3       1.00      1.00      1.00        24
           4       1.00      0.95      0.97        37
           5       1.00      1.00      1.00        29
           6       0.96      0.96      0.96        24
           7       1.00      1.00      1.00        34
           8       1.00      1.00      1.00        21
           9       0.97      1.00      0.99        34

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300

Test accuracy of the SNN: 0.99


In [11]:
model = tf.keras.models.load_model('CIFAR_v2.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.9937 - loss: 0.0186


In [12]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.9937
Test accuracy of the SNN: 0.99
